In [ ]:
import DeepFMKit.core as dfm

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import scipy.constants as sc

In [ ]:
dff = dfm.DeepFitFramework()

label = "sim"
dff.new_sim(label)
dff.sims[label].m = 6.0
dff.sims[label].f_mod = 1000
dff.sims[label].f_samp = int(200e3)
dff.sims[label].f_n = 1e6
dff.sims[label].arml_mod_f = 1.0
dff.sims[label].arml_mod_amp = 1e-9
dff.sims[label].arml_mod_n = 1e-12
dff.sims[label].fit_n = 10
dff.simulate(label, n_seconds=10, simulate="static")

dff.fit(label=label, fit_label='ideal modulation')

# Add the distortion parameters
dff.sims[label].df_2nd_harmonic_frac = 0.2  # Add 2% second-harmonic distortion
dff.sims[label].df_2nd_harmonic_phase = np.pi / 4 # Give it a relative phase
dff.simulate(label, n_seconds=10, simulate="static")

dff.fit(label=label, fit_label='distorted modulation')

In [ ]:
# Plot results
ax = dff.plot(labels=['ideal modulation', 'distorted modulation'])
ax[0].legend()
plt.show()

In [ ]:
def analyze_bias_vs_nonlinearity(
    m_true=15.5, 
    ndata=15, 
    distortion_range=np.linspace(0, 0.05, 11), 
    n_phase_trials=100
):
    """
    Analyzes the bias in 'm' due to harmonic distortion using a Monte Carlo
    approach over the distortion's phase.

    For each level of distortion amplitude, this function runs multiple trials,
    each with a new random phase for the 2nd harmonic. It then computes the
    mean and standard deviation of the resulting bias, providing a robust
    statistical picture of the systematic error's impact.

    Parameters
    ----------
    m_true : float, optional
        The ground-truth modulation depth to test.
    ndata : int, optional
        The number of harmonics to use in the fit.
    distortion_range : array_like, optional
        The range of fractional distortion amplitudes (epsilon) to test.
    n_phase_trials : int, optional
        The number of random phase trials to run for each distortion amplitude.
    """
    # --- 1. Setup ---
    print("="*60)
    print("Robust Analysis of Bias due to Modulation Non-Linearity")
    print("Running Monte Carlo simulation over 2nd harmonic phase...")
    print(f"Parameters: m_true={m_true}, ndata={ndata}, n_phase_trials={n_phase_trials}")
    print("="*60)

    dff = dfm.DeepFitFramework()
    label_base = "nl_robust_test"
    
    # These lists will store the final statistics for plotting
    mean_bias_list = []
    std_dev_bias_list = []

    # --- 2. Outer Loop: Iterate over distortion amplitude ---
    for eps in tqdm(distortion_range, desc="Distortion Level"):
        biases_for_this_epsilon = []
        
        # --- 3. Inner Loop: Monte Carlo over distortion phase ---
        for i in range(n_phase_trials):
            label = f"{label_base}_{i}"
            dff.new_sim(label)
            sim_config = dff.sims[label]
            # Configure the simulation for this specific trial
            sim_config.m = m_true
            sim_config.amp = 1.0
            sim_config.f_mod = 1000
            sim_config.f_samp = 200000
            sim_config.fit_n = 20
            # Set the distortion amplitude and a *random* phase for this trial
            sim_config.df_2nd_harmonic_frac = eps
            sim_config.df_2nd_harmonic_phase = np.random.uniform(0, 2 * np.pi)
            
            # Simulate a single, noiseless, distorted buffer
            dff.simulate(label, n_seconds=(sim_config.fit_n / sim_config.f_mod), simulate="static")
            
            # Fit the distorted data with the standard NLS fitter
            initial_guess = np.array([sim_config.amp, m_true, 0, 0])
            R, _, _ = dff.fit_init(label, sim_config.fit_n)
            fit_result = dff._fit_single_buffer(label, 0, R, ndata, initial_guess)
            
            bias = fit_result['m'] - m_true
            biases_for_this_epsilon.append(bias)

        # --- 4. Calculate statistics for this distortion level ---
        mean_bias_list.append(np.mean(biases_for_this_epsilon))
        std_dev_bias_list.append(np.std(biases_for_this_epsilon))

    # --- 5. Plotting ---
    mean_bias_array = np.array(mean_bias_list)
    std_dev_bias_array = np.array(std_dev_bias_list)
    
    # Convert 'm' bias to absolute length bias in nanometers
    # We need to calculate the df corresponding to m_true for a representative interferometer
    # Let's assume a 1-meter arm length difference to get a realistic df
    delta_l_assumed = 1.0 
    df = (m_true * sc.c) / (2 * np.pi * delta_l_assumed)
    m_to_length_factor = (sc.c / (2 * np.pi * df)) * 1e9 # Factor to convert delta_m to delta_l_nm
    
    length_bias_mean = m_to_length_factor * mean_bias_array
    length_bias_std = m_to_length_factor * std_dev_bias_array
    
    wavelength_nm = 1550 # Assuming a standard telecom wavelength
    ambiguity_limit_nm = wavelength_nm / 2

    fig, ax = plt.subplots(figsize=(12, 7))
    
    # Plot the mean bias
    ax.plot(distortion_range * 100, length_bias_mean, 'o-', color='tab:red', linewidth=2.5, label='Mean Bias')
    
    # Plot the shaded region for +/- 1 standard deviation
    ax.fill_between(distortion_range * 100, 
                    length_bias_mean - length_bias_std, 
                    length_bias_mean + length_bias_std, 
                    color='tab:red', alpha=0.2, label='±1 Std. Dev. of Bias (from phase)')

    # Plot the ambiguity limit
    ax.axhline(ambiguity_limit_nm, color='k', linestyle='--', linewidth=2, label=f'Ambiguity Limit ($\lambda_0/2$)')
    ax.axhline(-ambiguity_limit_nm, color='k', linestyle='--', linewidth=2)
    
    ax.set_xlabel('2nd Harmonic Distortion Amplitude (%)', fontsize=14)
    ax.set_ylabel('Absolute Length Error (nm)', fontsize=14)
    ax.set_title(f'Systematic Error from Modulation Non-Linearity (m_true = {m_true})', fontsize=16)
    ax.grid(True, which='both', linestyle=':')
    ax.legend(fontsize=12)
    plt.tight_layout()
    plt.show()

analyze_bias_vs_nonlinearity(m_true=15.5, ndata=15)